In [ ]:
# !pip install kagglehub
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("aladdinss/license-plate-digits-classification-dataset")

# print("Path to dataset files:", path)

In [1]:
from torch.nn import Sequential, Conv2d, MaxPool2d, Flatten, Linear, BatchNorm2d
import torch
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

In [2]:
print(torch.__version__)

2.9.1+cu128


In [3]:
print(torch.cuda.get_device_name(0))

NVIDIA GeForce RTX 3090


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
path = "/root/.cache/kagglehub/datasets/aladdinss/license-plate-digits-classification-dataset/versions/1/CNN letter Dataset/"

In [9]:
# load dataset (resize -> tensor so DataLoader can batch)
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])
dataset = ImageFolder(path, transform=transform)

In [10]:
num_classes = len(dataset.classes)
print(num_classes)

35


In [11]:
model = Sequential(
    # Depthwise-separable conv block
    Conv2d(3, 3, kernel_size=3, stride=1, padding=1, groups=3, bias=False),  # depthwise
    BatchNorm2d(3),
    torch.nn.ReLU(inplace=True),
    Conv2d(3, 32, kernel_size=1, bias=False),  # pointwise
    BatchNorm2d(32),
    torch.nn.ReLU(inplace=True),
    MaxPool2d(2),

    # Standard conv blocks
    Conv2d(32, 64, kernel_size=3, padding=1, bias=False),
    BatchNorm2d(64),
    torch.nn.ReLU(inplace=True),
    MaxPool2d(2),

    torch.nn.AdaptiveAvgPool2d((1, 1)),
    Flatten(),
    Linear(64, 128),
    torch.nn.ReLU(inplace=True),
    Linear(128, num_classes),
)

In [ ]:
from torch.utils.data import random_split

# Split dataset 80/20 train/val
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 40

In [15]:
def early_stopping(val_acc, best_val_acc, patience_counter, patience):
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        return True, best_val_acc, patience_counter
    return False, best_val_acc, patience_counter



In [16]:

best_val_acc = 0.0
patience_counter = 0
patience = 10

# Ensure model is on the same device as your batches (cuda/cpu)
model = model.to(device)

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

    train_loss = train_loss / len(train_loader)
    train_acc = 100 * train_correct / train_total

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_loader)
    val_acc = 100 * val_correct / val_total

    stop, best_val_acc, patience_counter = early_stopping(
        val_acc, best_val_acc, patience_counter, patience
    )
    if stop:
        print("Early stopping triggered")
        break

    print(f'Epoch [{epoch+1}/{num_epochs}]')
    print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
    print()
    print(f'Best Val Acc: {best_val_acc:.2f}%, Patience Counter: {patience_counter}/{patience}')

torch.save(model, 'cnn_model.pth')
print(f'Model saved - Final Val Accuracy: {val_acc:.2f}%')

Epoch [1/50]
  Train Loss: 2.1381, Train Acc: 32.87%
  Val Loss: 3.1243, Val Acc: 15.39%

Best Val Acc: 15.39%, Patience Counter: 0/10
Epoch [2/50]
  Train Loss: 1.8449, Train Acc: 43.34%
  Val Loss: 1.7796, Val Acc: 44.63%

Best Val Acc: 44.63%, Patience Counter: 0/10
Epoch [3/50]
  Train Loss: 1.6591, Train Acc: 49.16%
  Val Loss: 1.6229, Val Acc: 52.14%

Best Val Acc: 52.14%, Patience Counter: 0/10
Epoch [4/50]
  Train Loss: 1.4914, Train Acc: 55.13%
  Val Loss: 1.8933, Val Acc: 38.69%

Best Val Acc: 52.14%, Patience Counter: 1/10
Epoch [5/50]
  Train Loss: 1.3701, Train Acc: 59.05%
  Val Loss: 1.4441, Val Acc: 53.41%

Best Val Acc: 53.41%, Patience Counter: 0/10
Epoch [6/50]
  Train Loss: 1.2466, Train Acc: 63.24%
  Val Loss: 1.6924, Val Acc: 44.39%

Best Val Acc: 53.41%, Patience Counter: 1/10
Epoch [7/50]
  Train Loss: 1.1514, Train Acc: 66.25%
  Val Loss: 1.0389, Val Acc: 71.48%

Best Val Acc: 71.48%, Patience Counter: 0/10
Epoch [8/50]
  Train Loss: 1.0689, Train Acc: 68.77%
  